# Pytorch Guide

In [21]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

## Layers and Modules

In [2]:
# 1 hidden layer of size 256 w/ReLU, 10 unit output layer
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10)) # with LazyLinear, just need to specify output dimension
X = torch.rand(2,20)
net(X).shape

/home/will/ML-course-venv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

In [3]:
"""Custom PyTorch module. All modules must do the following:
- Ingest input data as arguments for forward propagation
- Return an output from forward propagation
- Calculate gradient of the output w.r.t. the input, usually automatic
- Store and allow access to necessary parameters for forward propagation
- Initialize parameters as needed"""

class MLP(nn.Module): # Module version of MLP defined above
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
net(X).shape

torch.Size([2, 10])

In [4]:
# Custom implementation of nn.Sequential
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx),module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

## Parameter Management

In [5]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [8]:
# can iterate over layers in nn.Sequential as a list
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.3494,  0.1806, -0.2886,  0.2697,  0.2727, -0.1644,  0.2401, -0.0132]])),
             ('bias', tensor([-0.1651]))])

In [9]:
# access specific parameter
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.1651]))

In [11]:
# backpropagation hasn't been passed yet
net[2].weight.grad == None

True

In [12]:
# All parameters at once
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

In [13]:
# Tied parameters
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## Parameter Initialization

In [14]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape


torch.Size([2, 1])

In [16]:
# initialize weights from a gaussian distribution
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([0.0130, 0.0073, 0.0055, 0.0131]), tensor(0.))

In [17]:
# initialize to a constant
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [18]:
# initialize different blocks differently
def init_xavier(module): # sample weights from gaussian with zero mean and variance 2/(n_in+n_out)
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1638, -0.3539,  0.4808, -0.6218])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [19]:
# initialization can be custom
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  8.4874,  6.2292,  5.2923],
        [-6.6818, -8.6058, -6.4482,  0.0000]], grad_fn=<SliceBackward0>)

In [20]:
# parameters can also be set directly
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  9.4874,  7.2292,  6.2923])

## Custom Layers

In [24]:
# Layer without parameters
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.tensor([1.0,2.0,3.0,4.0,5.0]))

tensor([-2., -1.,  0.,  1.,  2.])

In [25]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

/home/will/ML-course-venv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(3.7253e-09, grad_fn=<MeanBackward0>)

In [26]:
# Layer with parameters
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(6,2)
linear.weight

Parameter containing:
tensor([[ 0.4693,  1.3806],
        [ 0.5586, -3.1307],
        [ 0.1918, -0.7010],
        [ 0.9296,  0.1489],
        [ 1.4508,  0.2966],
        [-1.4287, -0.1476]], requires_grad=True)

## File I/O

In [27]:
# Loading and saving tensors
x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [28]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [29]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [30]:
# Load and save model parameters
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

torch.save(net.state_dict(), 'mlp.params') # save the parameters

clone = MLP() # have to instantiate new model
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [31]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])